In [ ]:
import numpy as np
import pandas as pd

import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
SINAN_PATH      = './../data/raw_data/SINAN_prep_05.csv'
MUNICIPIOS_PATH = './../data/municipios_prep_05.csv'
BOLSA_PATH = './../data/consolidada_bolsafamilia.csv'
INEP_PATH = './../data/consolidada_inep.csv'
ATLAS_PATH = './../data/atlas_desenvolvimento_humano_por_municipio.csv'
OCUPACOES_PATH  = './../data/cbo_ocupacoes.csv'
PNAD_PATH  = './../data/PNAD_consolidado.csv'
MUNIC_PATH = './../data/MUNIC_v2.csv'
SIM_PATH = './../data/SIM_consolidado.csv'
MAPBOX_TOKEN    = 'pk.eyJ1IjoibHVjYXNuc2VxIiwiYSI6ImNrb241dHZ0cTBpd2MycW5yMGp2enFtMmkifQ.N6NJGlWhG-iYrIJMQ1MVVw'

px.set_mapbox_access_token(MAPBOX_TOKEN)

### Base dos municípios

In [ ]:
municipios_df = pd.read_csv(MUNICIPIOS_PATH)

In [ ]:
municipios_df.head()

### Base SIM


In [ ]:
sim_df = pd.read_csv(SIM_PATH).drop(columns=['Unnamed: 0'])

In [ ]:
sim_df.head()

### Base MUNIC

In [ ]:
munic_df = pd.read_csv(MUNIC_PATH).drop(columns=['Unnamed: 0']).set_index('2013_A1')

In [ ]:
munic_df.head()

### Base do PNAD

In [ ]:
pnad_df = pd.read_csv(PNAD_PATH)
pnad_df = pnad_df.dropna().reset_index(drop=True).drop(columns=['Unnamed: 0'])
pnad_df = pnad_df.groupby('AGREGA').mean()

In [ ]:
pnad_df.head()

### Base do INEP

In [ ]:
inep_df = pd.read_csv(INEP_PATH)

In [ ]:
inep_df.shape

In [ ]:
inep_df.head()

### Base do Bolsa Familia

In [ ]:
bolsa_df = pd.read_csv(BOLSA_PATH)

In [ ]:
bolsa_df.shape

#### Bolsa colunas:

- renda_media_sum
- renda_media_mean
- renda_media_median
- renda_media_std
- numero_comodos_sum
- numero_comodos_mean
- numero_comodos_median
- numero_comodos_std
- numero_comodos_dorm_sum
- numero_comodos_dorm_mean
- numero_comodos_dorm_median
- numero_comodos_dorm_std

- share_agua_canalizada

In [ ]:
use_cols = ['renda_media_sum','renda_media_mean','renda_media_median',
            'renda_media_std','numero_comodos_sum','numero_comodos_mean',
            'numero_comodos_median','numero_comodos_std','numero_comodos_dorm_sum',
            'numero_comodos_dorm_mean','numero_comodos_dorm_median','numero_comodos_dorm_std',
            'share_agua_canalizada']

### Adicionando dados do Bolsa Familia

In [ ]:
df       = bolsa_df.set_index('2018_cd_ibge')
mun_df   = municipios_df.set_index('id')
mun_size = len(mun_df)

for cod_mun in tqdm(df.index):
    
    for year in [2013,2014,2015,2016,2017]:
        
        for col in use_cols:
        
            col_name = f'{year}_{col}'
            
            value = df.loc[cod_mun][col_name]
            
            col_name = f'{col}_{year}'
            
            mun_df.at[cod_mun, col_name] = value
            
municipios_df = mun_df.reset_index()

### Adicionando dados do PNAD

In [ ]:
mun_df = municipios_df.set_index('id')

pnad_col_map = {}
for column in pnad_df.columns:
    
    pnad_col_map[column] = f'pnad_{column.lower()}'

for index in tqdm(mun_df.index):
    
    mun_data  = mun_df.loc[index]
    
    
    pnad_data = pnad_df.loc[mun_data.uf_id]
    
    for column in pnad_df.columns:
        
        value    = pnad_data[column]
        col_name = pnad_col_map[column]
        
        mun_df.at[index, col_name] = value

municipios_df = mun_df.reset_index()

### Adicionando dados do MUNIC

In [ ]:
mu_df = munic_df.copy()

mu_df.head()

In [ ]:
cols = ['2013_A2', '2013_A3', '2013_A4', '2013_A5', '2013_A6', '2013_A7', '2013_A21', '2013_A27', '2013_A34', '2013_A229',
        '2013_A230', '2013_A232', '2013_A233', '2013_A234', '2013_A235', '2013_A236', '2013_A237', '2013_A238', '2013_A239',
        '2013_A240', '2013_A285', '2019_MTIC011', '2019_MTIC012', '2019_MTIC013', '2019_MTIC014', '2019_MTIC015',
        '2019_MTIC016', '2019_MTIC017', '2019_MGOV04', '2019_MSEG01', '2019_MSEG17.1', '2019_MDHU556']

nan_words = ['Recusa', 'Ignorada', 'Não informado', 'Não sabe informar', 'Não aplicável', 'Ignorado', 'Não informou']

feature_cols = ['2013_A2 ', '2013_A3', '2013_A5', '2013_A6', '2013_A7', '2013_A27', '2013_A34', '2013_A229', '2013_A230',
                '2013_A232', '2013_A234', '2013_A235', '2013_A236', '2013_A237', '2013_A238', '2013_A239', '2013_A285',
                '2019_MTIC011', '2019_MTIC012', '2019_MTIC013', '2019_MTIC014','2019_MTIC015','2019_MTIC016','2019_MTIC017',
                '2019_MGOV04','2019_MSEG01','2019_MSEG17.1','2019_MDHU556']

mu_df = munic_df[cols]

for index in tqdm(mu_df.index):
    
    mu_data = mu_df.loc[index]
    
    for col in mu_df.columns:
        
        value = mu_data[col]
        
        if value in nan_words:
            
            mu_df.at[index, col] = None

mu_df.head()

In [ ]:
mun_df = municipios_df.set_index('sinan_id')

munic_col_map = {}

for column in mu_df.columns:
    
    munic_col_map[column] = f'munic_{column.lower()}'
    
    if '2013' in munic_col_map[column]:
        
        munic_col_map[column] = munic_col_map[column].replace('2013', 'y13')
        
    if '2019' in munic_col_map[column]:
        
        munic_col_map[column] = munic_col_map[column].replace('2019', 'y19')

for mun_id in tqdm(mun_df.index):
    
    mun_data   = mun_df.loc[mun_id]
    munic_data = mu_df.loc[mun_id]
    
    for column in mu_df.columns:
        
        value    = munic_data[column]
        col_name = munic_col_map[column]
        
        mun_df.at[mun_id, col_name] = value

municipios_df = mun_df.reset_index()

municipios_df.head()

In [ ]:
# municipios_df.to_csv('municipios_prep_05.csv', index=False)

### Adicionando dados do INEP

#### Selecionando colunas

In [ ]:
temp_cols = []
icg_group_values = [f'ICG_{num}' for num in range(1, 7)]

for col in inep_df.columns:
    
    for icg in icg_group_values:
        
        if icg in col:
            temp_cols.append(col)

select_cols = []
for col in temp_cols:
    
    if inep_df.loc[1, col] == 'Total' and inep_df.loc[2, col] == 'Total':
        select_cols.append(col)
    

cols_map = {}
for i, icg_group in enumerate(icg_group_values):
    
    cols_map[icg_group] = {}
    icg_cols = []
    for col in select_cols:
        if icg_group in col:
            icg_cols.append(col)
    
    for j, year in enumerate([2013, 2014, 2015, 2016, 2017]):
        
        cols_map[icg_group][year] = icg_cols[j]
    
cols_map

#### Adicionando dados

In [ ]:
df      = inep_df.set_index('COD_MUNICIPIO')
mun_df  = municipios_df.set_index('id')

icg_group_values = [f'ICG_{num}' for num in range(1, 7)]
years = [2013, 2014, 2015, 2016, 2017]

for cod_mun in tqdm(df.index):
    
    if np.isnan(cod_mun): continue

    for icg_group in icg_group_values:

        for year in [2013, 2014, 2015, 2016, 2017]:

            col = cols_map[icg_group][year]
            value = df.loc[cod_mun][col]
            
            col_name = f'{icg_group.lower()}_total_{year}'
            
            mun_df.at[cod_mun, col_name] = value
            
municipios_df = mun_df.reset_index()

### Dados SIM

In [ ]:
cols = []

for year in [2013,2015,2015,2016,2017]:
    
    y_str = str(year)
    
    for col in sim_df.columns:
        if y_str in col and 'pop' not in col and col not in cols:
            cols.append(col)

rename_cols_map = {}
for col in cols:
    
    splits = col.split('_')
    
    year     = splits[0]
    new_name = '_'.join(splits[1:])
    rename_cols_map[col] = f'{new_name}_{year}'

sim_df2 = sim_df.set_index('COMUNSVOIM')
sim_df2 = sim_df2[cols].rename(rename_cols_map, axis=1)

sim_df2['uf_id'] = sim_df['COMUNSVOIM'].apply(lambda x : int(str(x)[:2])).values

group = sim_df2.groupby('uf_id').mean()

sim_df2 = sim_df2.drop(columns=['uf_id'])

sim_df2.head()

In [ ]:
group.head()

In [ ]:
mun_df = municipios_df.set_index('sinan_id')

for mun_id in tqdm(mun_df.index):
    
    mun_data = mun_df.loc[mun_id]
    
    for col in sim_df2.columns:
        
        if mun_id in sim_df2.index:
            
            mun_df.at[mun_id, col] = sim_df2.loc[mun_id][col]
            
        else:
            
            uf_id = mun_data['uf_id']
            
            mun_df.at[mun_id, col] = group.loc[uf_id][col]

mun_df = mun_df.reset_index()

mun_df.head()

### Adicionando cluster

In [ ]:
def get_encoder(values):
    
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(values.reshape(-1, 1))
    
    return encoded, encoder

In [ ]:
try:
    values_df = municipios_df.drop(columns=['id', 'nome', 'uf_nome', 'uf_id', 'sinan_id', 'latitude', 'longitude', 'cluster', 'cluster_id'])
except:
    values_df = municipios_df.drop(columns=['id', 'nome', 'uf_nome', 'uf_id', 'sinan_id', 'latitude', 'longitude'])

encoders = {}
encodeds = {}
feature_columns = ['uf', 'regiao']

for column in feature_columns:
    
    encodeds[column], encoders[column] = get_encoder(values_df[column].values)
    values_df[f'{column}_ID'] = encodeds[column]

values_df = values_df.fillna(-1)

In [ ]:
scaler = MinMaxScaler()

values        = values_df.drop(columns=feature_columns).values
scaled_values = scaler.fit_transform(values)

# Run the Kmeans algorithm and get the index of data points clusters
sse = []
n_clusters_list = list(range(1, 10))

for n_clusters in n_clusters_list:
    km = KMeans(n_clusters=n_clusters)
    km.fit(scaled_values)
    sse.append(km.inertia_)

cluster_eval_df = pd.DataFrame({'n_clusters': n_clusters_list, 'inertia': sse})

fig = px.line(
    cluster_eval_df,
    x='n_clusters', y='inertia', 
    title='Cluster evaluation')

fig.show()  

In [ ]:
clusterizer = KMeans(n_clusters = 3)
scaler = MinMaxScaler()

values = values_df.drop(columns=feature_columns).values
scaled_values = scaler.fit_transform(values)

clusterizer = clusterizer.fit(scaled_values)

municipios_df['cluster']    = clusterizer.predict(scaled_values)

#### Visualização de correlações

In [ ]:
keep_columns   = list(municipios_df.columns)
remove_columns = ['id','nome','uf_nome','uf_id','uf','regiao','latitude','longitude', 'sinan_id', 'cluster']

for c in remove_columns:
    try:
        keep_columns.remove(c)
    except:
        continue

In [ ]:
df    = pd.DataFrame()
years = [2013, 2014, 2015, 2016, 2017]
num_years = len(years)

for index in tqdm(municipios_df.index):
    
    mun_data = municipios_df.loc[index]
    
    temp_df = pd.DataFrame()
    
    temp_df['year'] = years
    
    for col in remove_columns:
        temp_df[col]   = [mun_data[col]] * num_years
    
    for col in keep_columns:
        
        col = col.split('_201')[0]
        
        values = []
        
        for year in years:

            col_name = f'{col}_{year}'
            
            value = mun_data[col_name]
            values.append(value)
            
        temp_df[col] = values
        
    df = pd.concat((df, temp_df))    

In [ ]:
fig = px.scatter_matrix(df,
    dimensions=["icg_2_total", "icg_4_total", 'icg_6_total', 'renda_media_mean', 'denun_maioridade_relat', 'denun_sexismo_relat', 'unit_relat', 'denun_relat', 'year'],
    color="cluster", height=1000)
fig.show()